<a href="https://colab.research.google.com/github/capozzolicaluan/MSWEPtest/blob/main/extraiMSWEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Limpar todas as variáveis
from IPython import get_ipython
get_ipython().magic('reset -sf')

# Código para reiniciar o ambiente do Colab
import os
os.kill(os.getpid(), 9)

In [1]:
pip install netcdf4

In [2]:
# Importando as bibliotecas necessárias
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta
import gc
import os as os
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import data_table
import math
from matplotlib.colors import TABLEAU_COLORS, same_color
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import glob
import xarray as xr
from tqdm import tqdm # para monitorar o progresso do processo


# Liberar memória manualmente
_ = gc.collect()

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
caminho_teste = "/content/drive/MyDrive/MSWEP_V280/NRT/3hourly"

In [5]:
lista_estacoes = pd.read_excel("/content/drive/MyDrive/estacoes_mswep.xlsx", header = None, names=['codigo','lat','lon'])

In [6]:
arquivos=os.listdir(caminho_teste)

In [7]:
# Filtra apenas os arquivos que começam com os anos desejados
anos_validos = ("2020","2021","2022","2023","2024")
arquivos_filtrados = [arq for arq in arquivos if arq.startswith(anos_validos)]

In [8]:
lista_arquivos = [os.path.join(caminho_teste, nome) for nome in arquivos_filtrados]

In [12]:
lista_arquivos = lista_arquivos[0:100]

In [9]:
# Pasta de saída (cria se não existir)
pasta_saida = "/content/drive/MyDrive/Colab/txts"
os.makedirs(pasta_saida, exist_ok=True)

In [10]:
for arquivo in tqdm(lista_arquivos):
    try:
        with xr.open_dataset(arquivo) as ds:
            linhas = []

            for i in range(len(lista_estacoes)):
                est = lista_estacoes.iloc[i]
                lat, lon, nome = est['lat'], est['lon'], est['codigo']

                valor = ds['precipitation'].sel(lat=lat, lon=lon, method='nearest').values.item()
                linha = f"{nome},{lat},{lon},{valor:.4f}"
                linhas.append(linha)

        nome_base = os.path.basename(arquivo).replace(".nc", "")
        caminho_saida = os.path.join(pasta_saida, f"{nome_base}.txt")

        with open(caminho_saida, "w") as f:
            f.write("nome,lat,lon,chuva\n")
            f.write("\n".join(linhas))

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {e}")

    gc.collect()

100%|██████████| 11987/11987 [5:58:23<00:00,  1.79s/it]


In [ ]:
lista_chuva = []

for arquivo in lista_arquivos:
    try:
        with xr.open_dataset(arquivo) as ds:
            # Lista para os valores de cada estação neste arquivo
            precipitation_at_point = [None] * len(lista_estacoes)

            for i in range(len(lista_estacoes)):
                lat = lista_estacoes.iloc[i]['lat']
                lon = lista_estacoes.iloc[i]['lon']

                # Seleciona valor de chuva mais próximo e converte para número
                valor = ds['precipitation'].sel(lat=lat, lon=lon, method='nearest').values.item()
                precipitation_at_point[i] = valor

            lista_chuva.append(precipitation_at_point)
            print(f"Processado: {arquivo}")

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {e}")

    gc.collect()